In [2]:
import fitz  
import re
import os
import csv
from datetime import datetime

def extraer_campos_verticales(pdf_path):
    doc = fitz.open(pdf_path)
    texto_completo = ""
    
    for pagina in doc:
        texto_completo += pagina.get_text("text") + "\n"
    
    doc.close()
    
    resultados = {}
    
    def extraer_valor_debajo(campo, texto, lineas_abajo=1):
        lineas = texto.split('\n')
        for i, linea in enumerate(lineas):
            if campo in linea:
                if i + lineas_abajo < len(lineas):
                    valor = lineas[i + lineas_abajo].strip()
                    if any(x in campo for x in ['Total', 'SALDO', 'IVA', 'Ret.', 'Percep.']):
                        valor = re.sub(r'[^\d.,]', '', valor).replace('.', '').replace(',', '.')
                        try:
                            return float(valor) if '.' in valor else int(valor)
                        except:
                            return valor
                    return valor
        return "No encontrado"

    campos_verticales = {
        "FECHA DE EMISION:": 22,
        "PAGADOR:": 22,
        "Nº DE CUIT:": 22,
        "TOTAL PRESENTADO $": 29,
        "TOTAL DESCUENTO": 29,
        "SALDO $": 29
    }
    
    for campo, lineas_abajo in campos_verticales.items():
        resultados[campo] = extraer_valor_debajo(campo, texto_completo, lineas_abajo)
    
    resultados["Razón Social"] = extraer_valor_debajo("Razón Social", texto_completo, 1)
    resultados["Establecimiento"] = extraer_valor_debajo("Establecimiento", texto_completo, 1)
    
    def extraer_valor_monetario(patron, texto):
        match = re.search(patron, texto)
        if match:
            valor = match.group(1).replace('.', '').replace(',', '.')
            try:
                return float(valor)
            except:
                return match.group(1).strip()
        return "No encontrado"
    
    
    resultados["IVA 21.00%"] = extraer_valor_monetario(r'IVA\s*21[.,]?00\s*%?\s*\$\s*([\d.,]+)', texto_completo)
    resultados["IVA 10,50 % Ley 25.063"] = extraer_valor_monetario(r'IVA\s*10[.,]?50\s*%\s*Ley\s*25[.,]?063\s*\$\s*([\d.,]+)', texto_completo)
    resultados["Ret.IB CAP. FED."] = extraer_valor_monetario(r'Ret.IB CAP.FED.\s*[\d.,]+\s*%?\s*\$\s*([\d.,]+)', texto_completo)
    resultados["Percep. AFIP"] = extraer_valor_monetario(r'Percep./Retenc.AFIP\s*[-]?\s*DGI\s*\$\s*([\d.,]+)', texto_completo)
    
    mapeo_nombres = {
        "FECHA DE EMISION:": "FECHA DE EMISION",
        "PAGADOR:": "PAGADOR",
        "Nº DE CUIT:": "Nº DE CUIT",
        "TOTAL DESCUENTO": "Total Descuento",
        "SALDO $": "Saldo",
        "TOTAL PRESENTADO $": "Total Presentado",
        "IVA 21.00%": "IVA 21%",
        "IVA 10,50 % Ley 25.063": "IVA 10,5%",
        "Ret.IB CAP. FED.": "Retención IB",
        "Percep. AFIP": "Percepción AFIP"
    }
    # la k rompe los huevos cuando se modifica el map. sostener el for de esta manera.
    return {mapeo_nombres.get(k, k): v for k, v in resultados.items()}



carpeta_pdfs = "C:/Users/gesti/Downloads/Tarjetas"
archivo_csv = "resultados_extracion.csv"

campos_csv = [
    "FECHA DE EMISION",
    "PAGADOR",
    "Nº DE CUIT",
    "Razón Social",
    "Establecimiento",
    "Total Presentado",
    "Total Descuento",
    "Saldo",
    "IVA 21%",
    "IVA 10,5%",
    "Retención IB",
    "Percepción AFIP",
    "Archivo PDF"  
]

with open(archivo_csv, mode='w', newline='', encoding='utf-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=campos_csv)
    writer.writeheader()
    
    for archivo in os.listdir(carpeta_pdfs):
        if archivo.lower().endswith('.pdf'):
            try:
                pdf_path = os.path.join(carpeta_pdfs, archivo)
                print(f"Procesando: {archivo}")
                
                datos = extraer_campos_verticales(pdf_path)
                datos["Archivo PDF"] = archivo
                
                writer.writerow(datos)
                
            except Exception as e:
                print(f"Error al procesar {archivo}: {str(e)}")
                
                fila_error = {campo: f"ERROR: {str(e)}" for campo in campos_csv}
                fila_error["Archivo PDF"] = archivo
                writer.writerow(fila_error)

print(f"\nProceso completado. Resultados guardados en: {archivo_csv}")

Procesando: comprobante-MEN-COM00101075060073032025-03-31.pdf
Procesando: comprobante-MEN-COM00101075480073032025-03-31.pdf
Procesando: comprobante-MEN-COM00101075550073032025-03-31.pdf
Procesando: comprobante-MEN-COM00104498820073032025-03-31.pdf
Procesando: comprobante-MEN-COM00104498900073032025-03-31.pdf
Procesando: comprobante-MEN-COM00104499240073032025-03-31.pdf
Procesando: comprobante-MEN-COM00104499320073032025-03-31.pdf
Procesando: comprobante-MEN-COM00347939920073032025-03-31.pdf
Procesando: comprobante-MEN-COM00347940080073032025-03-31.pdf
Procesando: comprobante-MEN-COM00347940320073032025-03-31.pdf
Procesando: comprobante-MEN-COM00347940400073032025-03-31.pdf
Procesando: comprobante-MEN-COM00526685890073382025-03-31.pdf
Procesando: comprobante-MEN-COM00526686130073382025-03-31.pdf
Procesando: comprobante-MEN-COM00610517930071272025-03-31.pdf
Procesando: comprobante-MEN-COM00610518010071272025-03-31.pdf
Procesando: comprobante-MEN-COM00610518270071272025-03-31.pdf
Procesan

In [ ]:
pip install openpyxl